<a href="https://colab.research.google.com/github/ankita-03/big_data_proj/blob/main/watches_data2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<spark version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://pp

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-03-10 01:20:23--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.82MB/s    in 0.2s    

2022-03-10 01:20:23 (5.82 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Watches").getOrCreate()

In [4]:
from pyspark import SparkFiles
from pyspark.sql.types import IntegerType
# Load in tsv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df2 = spark.read.csv(SparkFiles.get('amazon_reviews_us_Watches_v1_00.tsv.gz'), sep='\t', header=True, inferSchema=True)

print(df2.count())
df2 = df2.dropna()
print(df2.count())
df2 = df2.dropDuplicates()
print(df2.count())

df2.show(10)


960872
960679
960679
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43765005|R1006OPQYUMAHS|B0000TIIZ0|     529990321|Timex Women's T47...|         Watches|          1|            4|          5|   N|                Y|   Not very happy...|[[ASIN:B0000TIIZ0...| 2008-03-05|
|         US|   44007561|R1010XUX2AIF10|B004KTK2M8|      38406392|Leather Watchband...|         Wat

In [5]:
df2.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [6]:
#Review #2
review2_df = df2.select(["review_id", "customer_id", "product_id","product_parent", "review_date"])
review2_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1006OPQYUMAHS|   43765005|B0000TIIZ0|     529990321| 2008-03-05|
|R1010XUX2AIF10|   44007561|B004KTK2M8|      38406392| 2013-04-11|
|R102DONPK6YEGN|   52805367|B00NHEU9U2|      33550162| 2015-06-23|
|R102N48HXFM9WY|   18449245|B000PSQXWG|     517553139| 2013-02-04|
|R1060WX8UX26JV|   18106128|B0045H3K0G|     989815375| 2013-04-15|
|R106D023EFXYHQ|   44398606|B007900Q9O|     159678416| 2014-09-27|
|R107WJY4V4IZ35|   44609431|B008DZVXVE|     541585709| 2012-12-21|
|R10B275CIJ1QSG|    6507909|B003WPUU0U|     386958467| 2013-10-23|
|R10BIT0LX7BBYD|   22307556|B00DBUVKP0|     578040401| 2015-06-21|
|R10C1EZ2LJLSKN|   18425532|B0088131JK|     326602930| 2015-04-24|
|R10EFKNXWK3DPX|   22150304|B00EXTRLMY|     325769741| 2015-08-05|
|R10EVIW43OIYWF|    1875215|B005SSWKMK|     259094190| 2014-04

In [7]:
from pyspark.sql.types import DateType
review2_df = review2_df.withColumn("review_date", review2_df["review_date"].cast(DateType()))
review2_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1006OPQYUMAHS|   43765005|B0000TIIZ0|     529990321| 2008-03-05|
|R1010XUX2AIF10|   44007561|B004KTK2M8|      38406392| 2013-04-11|
|R102DONPK6YEGN|   52805367|B00NHEU9U2|      33550162| 2015-06-23|
|R102N48HXFM9WY|   18449245|B000PSQXWG|     517553139| 2013-02-04|
|R1060WX8UX26JV|   18106128|B0045H3K0G|     989815375| 2013-04-15|
|R106D023EFXYHQ|   44398606|B007900Q9O|     159678416| 2014-09-27|
|R107WJY4V4IZ35|   44609431|B008DZVXVE|     541585709| 2012-12-21|
|R10B275CIJ1QSG|    6507909|B003WPUU0U|     386958467| 2013-10-23|
|R10BIT0LX7BBYD|   22307556|B00DBUVKP0|     578040401| 2015-06-21|
|R10C1EZ2LJLSKN|   18425532|B0088131JK|     326602930| 2015-04-24|
|R10EFKNXWK3DPX|   22150304|B00EXTRLMY|     325769741| 2015-08-05|
|R10EVIW43OIYWF|    1875215|B005SSWKMK|     259094190| 2014-04

In [8]:
review2_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [9]:
#Product #2
product2_df = df2.select(["product_id", "product_title"])
product2_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0000TIIZ0|Timex Women's T47...|
|B004KTK2M8|Leather Watchband...|
|B00NHEU9U2|Fanmis Men's Wate...|
|B000PSQXWG|Nautica Men's N14...|
|B0045H3K0G|Columbia Men's CT...|
|B007900Q9O|Citizen Quartz Go...|
|B008DZVXVE|Swiss Legend Men'...|
|B003WPUU0U|G-SHOCK The GA 10...|
|B00DBUVKP0|Citizen Men's JY8...|
|B0088131JK|ADIDAS Performanc...|
|B00EXTRLMY|Seiko Men's SNE27...|
|B005SSWKMK|Casio Men's PRW-2...|
|B000LVLBRY|NFL Kids' FF-CHI ...|
|B005VSJKFC|Modus GA460.1000....|
|B00150VV48|Casio General Men...|
|B0096IQZPQ|Stuhrling Origina...|
|B009XN2LSO|Xezo Unisex Archi...|
|B00020J16M|Timex Ironman Sle...|
|B001WAKQAQ|Skagen Ladies Wat...|
|B005I700ZK|Kenneth Cole New ...|
+----------+--------------------+
only showing top 20 rows



In [10]:
#Customer #2
customers2_df = df2.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers2_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12937860|             1|
|   14327548|             4|
|   22158149|             5|
|   12828562|             2|
|    4089580|             1|
|   20097895|             1|
|    2422277|             4|
|   35523976|             2|
|   14051742|             1|
|   14404180|             5|
|   12883856|             3|
|    9942583|             2|
|   38893812|             1|
|   22754948|             1|
|   51265055|             5|
|   49225180|             1|
|   16215726|             1|
|   39948072|            20|
|   14847189|             3|
|   17207294|             2|
+-----------+--------------+
only showing top 20 rows



In [11]:
from pyspark.sql.types import IntegerType
customers2_df = customers2_df.withColumn("customer_count", customers2_df["customer_count"].cast(IntegerType()))
customers2_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12937860|             1|
|   14327548|             4|
|   22158149|             5|
|   12828562|             2|
|    4089580|             1|
|   20097895|             1|
|    2422277|             4|
|   35523976|             2|
|   14051742|             1|
|   14404180|             5|
|   12883856|             3|
|    9942583|             2|
|   38893812|             1|
|   22754948|             1|
|   51265055|             5|
|   49225180|             1|
|   16215726|             1|
|   39948072|            20|
|   14847189|             3|
|   17207294|             2|
+-----------+--------------+
only showing top 20 rows



In [12]:
#Vine #2
vine2_df = df2.select(["review_id", "star_rating", "helpful_votes","total_votes", "vine"])
vine2_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1006OPQYUMAHS|          1|            4|          5|   N|
|R1010XUX2AIF10|          5|            0|          0|   N|
|R102DONPK6YEGN|          5|            0|          0|   N|
|R102N48HXFM9WY|          5|            0|          0|   N|
|R1060WX8UX26JV|          2|            0|          0|   N|
|R106D023EFXYHQ|          5|            0|          0|   N|
|R107WJY4V4IZ35|          5|            0|          0|   N|
|R10B275CIJ1QSG|          4|            0|          0|   N|
|R10BIT0LX7BBYD|          4|            1|          2|   N|
|R10C1EZ2LJLSKN|          5|            0|          0|   N|
|R10EFKNXWK3DPX|          5|            0|          2|   N|
|R10EVIW43OIYWF|          4|            2|          2|   N|
|R10EZ3YEUODPD0|          5|            0|          0|   N|
|R10HN9CWJDWYKL|          5|            

In [13]:
from getpass import getpass
password = getpass('Enter Database Password here')

mode = "append"
jdbc_url="jdbc:postgresql://database-1.cgsoovj0vnlg.us-west-2.rds.amazonaws.com:5432/database-1"
config = {"user":"postgres", "password":password, "driver" :"org.postgresql.Driver"}

Enter Database Password here··········


In [15]:
review2_df.write.jdbc(url="jdbc:postgresql://database-1.cgsoovj0vnlg.us-west-2.rds.amazonaws.com", table="review_id_table2", mode="append", properties= config)

In [ ]:
product2_df.write.jdbc(url="jdbc:postgresql://database-1.cgsoovj0vnlg.us-west-2.rds.amazonaws.com", table="products2", mode="append", properties= config)

In [ ]:
customers2_df.write.jdbc(url="jdbc:postgresql://database-1.cgsoovj0vnlg.us-west-2.rds.amazonaws.com", table="customers2", mode="append", properties= config)

In [ ]:
vine2_df.write.jdbc(url="jdbc:postgresql://database-1.cgsoovj0vnlg.us-west-2.rds.amazonaws.com", table="vine_table2", mode="append", properties= config)